In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta

In [2]:
customers = pd.read_csv("data/task_DE_new_customers.csv")

In [3]:
customers.head()

,customer_id,customer_name,mobile_number,region
0,CUST-001,Aarav Mehta,9123456781,West
1,CUST-002,Neha Sharma,9123456782,North
2,CUST-003,Rohan Gupta,9123456783,South
3,CUST-004,Priya Iyer,9123456784,West
4,CUST-005,Kabir Singh,9123456785,Central


In [4]:
tree = ET.parse("data/task_DE_new_orders.xml")
root = tree.getroot()

In [5]:
orders = []
for order in root.findall("order"):
    orders.append({
        "order_id": order.find("order_id").text,
        "mobile_number": order.find("mobile_number").text,
        "order_date_time": order.find("order_date_time").text,
        "sku_id": order.find("sku_id").text,
        "sku_count": int(order.find("sku_count").text),
        "total_amount": float(order.find("total_amount").text)
    })

In [6]:
orders_df = pd.DataFrame(orders)

In [7]:
orders_df

,order_id,mobile_number,order_date_time,sku_id,sku_count,total_amount
0,ORD-2025-0001,9123456781,2025-10-12T09:15:32,SKU-1001,2,7450.0
1,ORD-2025-0001,9123456781,2025-10-12T09:15:32,SKU-1004,1,7450.0
2,ORD-2025-0001,9123456781,2025-10-12T09:15:32,SKU-1010,3,7450.0
3,ORD-2025-0002,9123456781,2025-11-01T14:22:10,SKU-1002,1,5299.0
4,ORD-2025-0002,9123456781,2025-11-01T14:22:10,SKU-1007,2,5299.0
5,ORD-2025-0002,9123456781,2025-11-01T14:22:10,SKU-1011,1,5299.0
6,ORD-2025-0003,9123456782,2025-09-30T18:45:55,SKU-1003,4,8930.0
7,ORD-2025-0003,9123456782,2025-09-30T18:45:55,SKU-1008,1,8930.0
8,ORD-2025-0003,9123456782,2025-09-30T18:45:55,SKU-1010,2,8930.0
9,ORD-2025-0003,9123456782,2025-09-30T18:45:55,SKU-1012,1,8930.0


In [8]:
orders_df.dtypes

order_id            object
mobile_number       object
order_date_time     object
sku_id              object
sku_count            int64
total_amount       float64
dtype: object

In [9]:
orders_df["order_date_time"] = pd.to_datetime(orders_df["order_date_time"], errors="coerce")

In [10]:
orders_df

,order_id,mobile_number,order_date_time,sku_id,sku_count,total_amount
0,ORD-2025-0001,9123456781,2025-10-12 09:15:32,SKU-1001,2,7450.0
1,ORD-2025-0001,9123456781,2025-10-12 09:15:32,SKU-1004,1,7450.0
2,ORD-2025-0001,9123456781,2025-10-12 09:15:32,SKU-1010,3,7450.0
3,ORD-2025-0002,9123456781,2025-11-01 14:22:10,SKU-1002,1,5299.0
4,ORD-2025-0002,9123456781,2025-11-01 14:22:10,SKU-1007,2,5299.0
5,ORD-2025-0002,9123456781,2025-11-01 14:22:10,SKU-1011,1,5299.0
6,ORD-2025-0003,9123456782,2025-09-30 18:45:55,SKU-1003,4,8930.0
7,ORD-2025-0003,9123456782,2025-09-30 18:45:55,SKU-1008,1,8930.0
8,ORD-2025-0003,9123456782,2025-09-30 18:45:55,SKU-1010,2,8930.0
9,ORD-2025-0003,9123456782,2025-09-30 18:45:55,SKU-1012,1,8930.0


In [11]:
customers.isna().sum()

customer_id      0
customer_name    0
mobile_number    0
region           0
dtype: int64

In [12]:
orders_df.isna().sum()

order_id           0
mobile_number      0
order_date_time    0
sku_id             0
sku_count          0
total_amount       0
dtype: int64

In [13]:
orders_df.dtypes

order_id                   object
mobile_number              object
order_date_time    datetime64[ns]
sku_id                     object
sku_count                   int64
total_amount              float64
dtype: object

### KPI 

In [15]:
customers["mobile_number"] = customers["mobile_number"].astype(str)
orders_df["mobile_number"] = orders_df["mobile_number"].astype(str)

data = pd.merge(orders_df, customers, on="mobile_number", how="left")

##### KPI 1: Repeat Customers

In [17]:
repeat_customers = (
    data.groupby("mobile_number")["order_id"]
    .nunique()
    .reset_index(name="order_count")
    .query("order_count > 1")
)

In [24]:
repeat_customers 

,mobile_number,order_count
0,9123456781,2


##### KPI 2: Monthly Order Trends

In [18]:
data["month"] = data["order_date_time"].dt.to_period("M")
monthly_trends = (
    data.groupby("month")["order_id"]
    .nunique()
    .reset_index(name="total_orders")
)

In [23]:
monthly_trends

,month,total_orders
0,2025-09,1
1,2025-10,1
2,2025-11,1


##### KPI 3: Regional Revenue

In [19]:
regional_revenue = (
    data.groupby("region")["total_amount"]
    .sum()
    .reset_index()
)

In [22]:
regional_revenue

,region,total_amount
0,North,35720.0
1,West,38247.0


##### KPI 4: Top Customers (Last 30 Days)

In [20]:
last_30_days = datetime.now() - timedelta(days=30)
recent_orders = data[data["order_date_time"] >= last_30_days]
top_customers = (
    recent_orders.groupby("customer_name")["total_amount"]
    .sum()
    .reset_index()
    .sort_values(by="total_amount", ascending=False)
    .head(5)
)

In [21]:
top_customers

,customer_name,total_amount
0,Aarav Mehta,38247.0


In [25]:
output_folder = "output/"
repeat_customers.to_csv(output_folder + "repeat_customers.csv", index=False)
monthly_trends.to_csv(output_folder + "monthly_trends.csv", index=False)
regional_revenue.to_csv(output_folder + "regional_revenue.csv", index=False)
top_customers.to_csv(output_folder + "top_customers.csv", index=False)

In [26]:
data.to_csv(output_folder + "data.csv", index=False)